# 9 - Caracterizacion Kmeans

**AUTOR: Fabrizio Ramirez Cutimbo**

**OBJETIVO**

+ Evaluar los Clusters creados en función de la métrica "Location Quotient"
+ Seleccionar los Clsuters con mejor puntuación
+ Describir con estadística descriptiva los Clusters
---

In [26]:
# Librerias de analisis de datos
import pandas as pd
import numpy as np

# Libreiras de visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
# Establecere formato de 2 decimales
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None) 

pd_companias = pd.read_csv('Datasets Procesados/directorio_empresas_final_clusterizado.csv', dtype={'RUC': 'object'})

In [28]:
print(pd_companias.shape)
pd_companias.sample(2)

(37884, 77)


,EXPEDIENTE,RUC,SITUACION_LEGAL,FECHA_CONSTITUCION,TIPO,PAIS,REGION,PROVINCIA,CANTON,CIUDAD,...,IF_MARGEN_OPERCIONAL,IF_MARGEN_NETO,IF_ROE,IF_ROA,Set10_PCA1,Set10_PCA2,CLUSTERS_KMEANS_SET_10,Set4_UMAP1,Set4_UMAP2,CLUSTER_DBSCAN_SET_4
34785,738113,1793170382001,ACTIVA,16/07/2021,SOCIEDAD POR ACCIONES SIMPLIFICADA,ECUADOR,SIERRA,PICHINCHA,QUITO,TUMBACO,...,0.05,0.00,0.00,0.00,-0.55,0.08,5,12.68,7.75,3
24970,344863,1793204450001,ACTIVA,17/02/2023,SOCIEDAD POR ACCIONES SIMPLIFICADA,ECUADOR,SIERRA,PICHINCHA,QUITO,NAYÓN,...,0.86,0.00,0.00,0.00,-1.13,0.35,0,18.74,-0.06,13


In [29]:
pd_companias.columns

Index(['EXPEDIENTE', 'RUC', 'SITUACION_LEGAL', 'FECHA_CONSTITUCION', 'TIPO',
       'PAIS', 'REGION', 'PROVINCIA', 'CANTON', 'CIUDAD', 'CIIU_NIVEL_1',
       'CIIU_NIVEL_1_DESC', 'CIIU_NIVEL_3', 'CIIU_NIVEL_3_DESC',
       'CIIU_NIVEL_4', 'CIIU_NIVEL_4_DESC', 'CIIU_NIVEL_6',
       'CIIU_NIVEL_6_DESC', 'ACTIVO_2022', 'ACTIVO_CORRIENTE_2022',
       'INVENTARIOS_2022', 'ACTIVOS_NO_CORRIENTES_2022', 'PASIVO_2022',
       'PASIVO_CORRIENTE_2022', 'PASIVO_NO_CORRIENTE_2022',
       'PATRIMONIO_NETO_2022', 'INGRESOS_ACTIVIDADES_ORDINARIAS_2022',
       'GANANCIA_BRUTA_2022', 'OTROS_INGRESOS_2022',
       'COSTO_VENTAS_PRODUCCION_2022', 'GASTOS_2022',
       'UTILIDAD_OPERATIVA_2022', 'GANACIA_PERDIDA_ANTES_IR_2022',
       'IMPUESTO_RENTA_2022', 'UTILIDAD_NETA_2022', 'ACTIVO_2023',
       'ACTIVO_CORRIENTE_2023', 'INVENTARIOS_2023',
       'ACTIVOS_NO_CORRIENTES_2023', 'PASIVO_2023', 'PASIVO_CORRIENTE_2023',
       'PASIVO_NO_CORRIENTE_2023', 'PATRIMONIO_NETO_2023',
       'INGRESOS_ACTIVID

In [30]:
# Reemplazar valores por fomato correcto
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('MICRO', 'Micro')
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('PEQUENA', 'Pequeña')
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('MEDIANA', 'Mediana')
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('GRANDE', 'Grande')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.capitalize()
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Galapagos', 'Galápagos')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Los rios', 'Los ríos')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Morona santiago', 'Morona Santiago')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Santo domingo de los tsachilas', 'Santo Domingo de los Tsáchilas')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Santa elena', 'Santa Elena')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Zamora chinchipe', 'Zamora Chinchipe')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Sucumbios', 'Sucumbíos')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('El oro', 'El Oro')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('COSTA', 'Costa')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('SIERRA', 'Sierra')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('AMAZONIA', 'Amazonía')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('GALÁPAGOS', 'Galápagos')


### Funciones 

In [31]:
def agrupar_por_cluster(dataset, columna_agrupacion):
    """
    Agrupa un dataset por una columna específica y calcula promedios, distribuciones
    de segmentos, regiones y actividades económicas.

    Args:
        dataset (pd.DataFrame): El conjunto de datos a agrupar.
        columna_agrupacion (str): La columna por la cual se agrupará el dataset.

    Returns:
        pd.DataFrame: Un DataFrame con los resultados del análisis por cluster.
    """
    # Agrupar por la columna especificada
    grouped_clusters = dataset.groupby(columna_agrupacion)

    # Crear lista para almacenar los resultados
    results = []

    # Iterar sobre cada grupo
    for cluster, group in grouped_clusters:

        #Contar el numero de empresas
        n_empresas = group.shape[0]        
        
        # ACTIVOS
        activos_stats = group['ACTIVO_2023'].agg(['mean', 'std', 'min', 'median', 'max'])
        activos_summary = (
            f"Mean: ${activos_stats['mean']:.1f}, "
            f"Std: {activos_stats['std']:.1f}, "
            # f"Min: ${ingresos_stats['min']:.1f}, "
            f"Median: ${activos_stats['median']:.1f} "
            # f"Max: ${ingresos_stats['max']:.2f}"
        )
        # PASIVO
        pasivos_stats = group['PASIVO_2023'].agg(['mean', 'std', 'min', 'median', 'max'])
        pasivos_summary = (
            f"Mean: ${pasivos_stats['mean']:.1f}, "
            f"Std: {pasivos_stats['std']:.1f}, "
            f"Median: ${pasivos_stats['median']:.1f} "
        )

        # Resumen estadístico para IF ROE
        ingresos_stats = group['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'].agg(['mean', 'std', 'min', 'median', 'max'])
        ingresos_summary = (
            f"Mean: ${ingresos_stats['mean']:.1f}, "
            f"Std: {ingresos_stats['std']:.1f} "
            f"Median: ${ingresos_stats['median']:.1f}, "
        )
        
        if_mg_neto_stats = group['IF_MARGEN_NETO'].agg(['mean', 'std', 'min', 'median', 'max'])
        if_mg_neto_summary = (
            f"Mean: {if_mg_neto_stats['mean']*100:.1f}%, "
            # f"Std: {if_mg_neto_stats['std']:.1f}, "            
            f"Median: {if_mg_neto_stats['median']*100:.1f}% "            
        )

        utilidad_neta_stats = group['UTILIDAD_NETA_2023'].agg(['mean', 'std', 'min', 'median', 'max'])
        percentil_75 = group['UTILIDAD_NETA_2023'].quantile(0.75)
        utilidad_neta_summary = (
            f"Mean: ${utilidad_neta_stats['mean']:.1f}, "
            f"Std: {utilidad_neta_stats['std']:.1f}, "
            # f"Min: ${utilidad_neta_stats['min']:.1f}, "
            f"Median: ${utilidad_neta_stats['median']:.1f} "
            # calcular el 75%
            # f"75%: ${percentil_75:.1f} "
            # f"Max: ${utilidad_neta_stats['max']:.2f}"
        )

        # Calcular distribución del segmento
        segment_distribution = group['SEGMENTO'].value_counts(normalize=True) * 100
        segment_distribution_str = ', '.join([f"{seg}: {perc:.1f}%" for seg, perc in segment_distribution.items()])

        # Calcular distribución de la región
        region_distribution = group['REGION'].value_counts(normalize=True) * 100
        region_distribution_str = ', '.join([f"{reg}: {perc:.1f}%" for reg, perc in region_distribution.items()])

        # Calcular distribución de CIIU_NIVEL_1
        ciiu_distribution = group['CIIU_NIVEL_1'].value_counts(normalize=True) * 100
        ciiu_distribution_str = ', '.join([f"{ciiu}: {perc:.1f}%" for ciiu, perc in ciiu_distribution.items()])

        # Almacenar resultados en la lista
        result_row = {
            columna_agrupacion: cluster,              
            'Activos': activos_summary,
            'Pasivos': pasivos_summary,
            'Ingresos': ingresos_summary,
            'IF Margen Neto': if_mg_neto_summary,
            'Utilidad Neta': utilidad_neta_summary,
            'Nro. Empresas': n_empresas,
            'Distribución Segmento': segment_distribution_str,
            'Distribución Región': region_distribution_str,
            'Distribución CIIU 1': ciiu_distribution_str
        }

        results.append(result_row)

    # Convertir resultados a DataFrame
    result_df = pd.DataFrame(results)

    # Reorganizar las columnas para que las distribuciones se muestren al final
    columnas_distribucion = ['Distribución Segmento', 'Distribución Región', 'Distribución CIIU 1']
    columnas_ordenadas = (
        [columna_agrupacion] +
        [col for col in result_df.columns if col not in columnas_distribucion and col != columna_agrupacion] +
        columnas_distribucion
    )

    # Reorganizar DataFrame
    result_df = result_df[columnas_ordenadas]

    return result_df

# Llamar a la función y mostrar el DataFrame resultante
result_df = agrupar_por_cluster(pd_companias, 'CLUSTERS_KMEANS_SET_10')

result_df.rename(columns={'CLUSTERS_KMEANS_SET_10': 'Cluster'}, inplace=True)

result_df


,Cluster,Activos,Pasivos,Ingresos,IF Margen Neto,Utilidad Neta,Nro. Empresas,Distribución Segmento,Distribución Región,Distribución CIIU 1
0,0,"Mean: $52700.1, Std: 136003.0, Median: $12523.1","Mean: $36131.8, Std: 128707.2, Median: $4196.4","Mean: $25971.3, Std: 44715.5 Median: $11812.2,","Mean: 9.7%, Median: 0.0%","Mean: $-1738.4, Std: 14232.0, Median: $0.0",19390,"Micro: 98.6%, Pequeña: 1.4%, Mediana: 0.0%","Sierra: 53.1%, Costa: 43.7%, ORIENTE: 2.2%, Galápagos: 1.0%","M: 39.7%, N: 19.3%, J: 13.3%, C: 11.1%, A: 7.4%, I: 5.5%, K: 3.7%"
1,1,"Mean: $5849588.8, Std: 2789920.4, Median: $5163320.3","Mean: $3430418.6, Std: 2201964.7, Median: $2851581.8","Mean: $6936350.8, Std: 3644366.1 Median: $5741721.7,","Mean: 3.9%, Median: 0.9%","Mean: $57753.4, Std: 66575.0, Median: $64012.8",327,"Grande: 66.4%, Mediana: 33.0%, Pequeña: 0.6%","Costa: 50.2%, Sierra: 46.5%, ORIENTE: 2.8%, Galápagos: 0.6%","A: 37.0%, C: 32.7%, M: 10.1%, N: 8.0%, K: 4.9%, J: 4.0%, I: 3.4%"
2,2,"Mean: $19670791.2, Std: 0.0, Median: $19670791.2","Mean: $11336951.3, Std: 0.0, Median: $11336951.3","Mean: $18348951.1, Std: 951987.1 Median: $18513770.4,","Mean: 3.6%, Median: 1.2%","Mean: $74326.4, Std: 69653.4, Median: $122394.4",67,Grande: 100.0%,"Costa: 56.7%, Sierra: 41.8%, Galápagos: 1.5%","C: 65.7%, A: 19.4%, N: 4.5%, J: 4.5%, M: 4.5%, K: 1.5%"
3,3,"Mean: $1424714.0, Std: 1526162.1, Median: $878212.4","Mean: $807485.5, Std: 1003461.6, Median: $464873.3","Mean: $1080073.1, Std: 698806.0 Median: $982747.2,","Mean: 5.1%, Median: 0.0%","Mean: $16230.4, Std: 49046.3, Median: $0.0",1827,"Mediana: 48.6%, Pequeña: 46.7%, Micro: 4.6%, Grande: 0.1%","Costa: 51.4%, Sierra: 46.9%, ORIENTE: 0.9%, Galápagos: 0.8%","C: 27.2%, A: 23.9%, M: 17.2%, N: 10.9%, K: 8.7%, J: 6.1%, I: 6.0%"
4,4,"Mean: $15447184.9, Std: 3529356.9, Median: $15613088.7","Mean: $8158863.0, Std: 2520353.1, Median: $8131500.1","Mean: $16020729.1, Std: 3183196.2 Median: $17245785.8,","Mean: 3.8%, Median: 1.8%","Mean: $80659.3, Std: 58807.4, Median: $122394.4",139,Grande: 100.0%,"Sierra: 51.1%, Costa: 48.2%, Galápagos: 0.7%","C: 55.4%, A: 28.8%, I: 5.8%, M: 4.3%, N: 2.9%, J: 2.2%, K: 0.7%"
5,5,"Mean: $240642.9, Std: 367621.6, Median: $125040.5","Mean: $143812.1, Std: 266261.9, Median: $64235.4","Mean: $241381.7, Std: 185498.2 Median: $187844.1,","Mean: 3.8%, Median: 0.0%","Mean: $3216.9, Std: 19237.0, Median: $0.0",8559,"Pequeña: 85.8%, Micro: 14.0%, Mediana: 0.2%","Sierra: 51.8%, Costa: 46.1%, ORIENTE: 1.4%, Galápagos: 0.7%","M: 31.9%, N: 18.5%, C: 14.4%, J: 11.5%, A: 11.1%, I: 7.7%, K: 5.0%"
6,6,"Mean: $2723331.4, Std: 1703184.4, Median: $2256455.5","Mean: $1551599.8, Std: 1186127.0, Median: $1235511.4","Mean: $3040545.4, Std: 1639115.6 Median: $2790512.9,","Mean: 7.0%, Median: 0.5%","Mean: $42491.5, Std: 62308.6, Median: $16651.5",696,"Mediana: 84.6%, Grande: 9.3%, Pequeña: 5.9%, Micro: 0.1%","Costa: 50.4%, Sierra: 48.4%, ORIENTE: 0.9%, Galápagos: 0.3%","C: 34.1%, A: 30.9%, M: 11.5%, K: 6.9%, N: 6.8%, J: 5.7%, I: 4.2%"
7,7,"Mean: $18042451.0, Std: 3291458.0, Median: $19670791.2","Mean: $10498074.6, Std: 1818854.7, Median: $11336951.3","Mean: $15333326.6, Std: 5170464.1 Median: $18513770.4,","Mean: 5.6%, Median: 0.0%","Mean: $18905.6, Std: 91759.8, Median: $0.0",69,"Grande: 92.8%, Mediana: 5.8%, Pequeña: 1.4%","Costa: 50.7%, Sierra: 44.9%, ORIENTE: 2.9%, Galápagos: 1.4%","C: 27.5%, A: 23.2%, K: 21.7%, J: 8.7%, N: 7.2%, M: 7.2%, I: 4.3%"
8,8,"Mean: $19670791.2, Std: 0.0, Median: $19670791.2","Mean: $11021046.6, Std: 1760012.0, Median: $11336951.3","Mean: $18066373.6, Std: 2543155.6 Median: $18513770.4,","Mean: 4.4%, Median: 1.4%","Mean: $59694.9, Std: 92420.5, Median: $122394.4",62,"Grande: 98.4%, Mediana: 1.6%","Costa: 58.1%, Sierra: 41.9%","C: 54.8%, A: 17.7%, K: 11.3%, J: 9.7%, N: 4.8%, M: 1.6%"
9,9,"Mean: $1785812.4, Std: 1353680.7, Median: $1402123.3","Mean: $986469.4, Std: 889173.5, Median: $722719.0","Mean: $1829861.4, Std: 953861.8 Median: $1710993.3,","Mean: 5.9%, Median: 0.2%","Mean: $29942.6, Std: 5

Exportar tabla resumen

In [32]:
result_df.to_csv(
    'Datasets Procesados\\Resultados\\Tabla_Resumen_Clusters_kmeans.csv', 
    sep=';',               # Separador de columnas
    decimal=',',           # Separador decimal
    index=False,           # Excluir índice
    encoding='utf-8-sig'   # Codificación compatible con Excel y Power BI
)

## Evaluación Location Quotient (LQ)

Función para cálculo de Location Quotient (LQ)

In [33]:
def calcular_location_quotient(dataset, metrica, provincia_col, cluster_col):
    """
    Calcula el Quotient Location (QL) para un dataset dado.

    Parámetros:
    - dataset: DataFrame de entrada que contiene los datos.
    - metrica: Columna que representa la métrica económica (e.g., Empleados UTILIDAD_NETA_2023).
    - provincia_col: Nombre de la columna que contiene las provincias (o región Menor).
    - cluster_col: Nombre de la columna que contiene los clusters (La Actividad Económica).

    Retorna:
    - DataFrame con columnas para provincia, cluster y el valor calculado de QL.
    """
    # Calcular totales por cluster y provincia
    # totales_clusters_provincias = dataset.groupby([provincia_col, cluster_col])[metrica].sum()
    totales_clusters_provincias = dataset.groupby([provincia_col, cluster_col])[metrica].agg(['sum', 'count']).reset_index()

    # Calcular totales por provincia
    totales_provincias = dataset.groupby(provincia_col)[metrica].sum()

    # Calcular totales por cluster a nivel nacional
    totales_clusters_nacional = dataset.groupby(cluster_col)[metrica].sum()

    # Calcular total nacional
    totales_nacional = dataset[metrica].sum()

    # Crear lista para almacenar los resultados
    lq_data = []
    
    for index, row in totales_clusters_provincias.iterrows():
        provincia = row[provincia_col]
        cluster = row[cluster_col]
        total_suma = row['sum']
        total_conteo = row['count']
        # Proporción del cluster dentro de la provincia
        proporcion_cluster_provincia = total_suma / totales_provincias[provincia]

        # Proporción del cluster a nivel nacional
        proporcion_cluster_region = totales_clusters_nacional[cluster] / totales_nacional

        # Se ha detectado que hay empresas que a pesar de tener ingresos no tienen resultados del ejercicio (utilidad, impuestos, etc)
        # Estos valores se establecen en 0, auqnue podrían establecerse en NaN para análisis futuros
        if proporcion_cluster_region == 0:
            lq = 0 # Si el Divisor es 0, no se realiza la división sino se establece directamente en 0
            
        else: 
            # Calcular QL
            lq = proporcion_cluster_provincia / proporcion_cluster_region

        # # Verificar si alguna proporción es un valor no numérico
        # if not np.isfinite(lq):
        #     print(f"Valor no numérico encontrado: {provincia} - {cluster}")
        #     print(f"Proporciones: {proporcion_cluster_provincia}, {proporcion_cluster_region}")
        #     lq = np.nan        

        # Agregar resultado a la lista
        lq_data.append({
            provincia_col: provincia,
            cluster_col: cluster,
            'Nro Empresas': total_conteo,
            'LQ': lq
        })

    # Convertir resultados a DataFrame
    lq_dataframe = pd.DataFrame(lq_data)

    # Ordenar por QL descendente
    lq_dataframe = lq_dataframe.sort_values(by='LQ', ascending=False)

    return lq_dataframe

Calculo de LQ Para Clusters

In [34]:
location_quotient_set_10 = calcular_location_quotient(dataset=pd_companias, metrica='INGRESOS_ACTIVIDADES_ORDINARIAS_2023', provincia_col='PROVINCIA', cluster_col='CLUSTERS_KMEANS_SET_10')
print(location_quotient_set_10.shape)
location_quotient_set_10.head(5)

# location_quotient_set_10_filtered = location_quotient_set_10[location_quotient_set_10['CLUSTERS_SET_10']>=5]
# print(location_quotient_set_10_filtered.shape)

# location_quotient_set_10_filtered.head(15)
# location_quotient_set_10[location_quotient_set_10['LQ']>2].head(10)

(303, 4)


,PROVINCIA,CLUSTERS_KMEANS_SET_10,Nro Empresas,LQ
35,Cañar,8,1,13.84
22,Bolivar,11,5,9.54
20,Bolivar,0,32,8.59
298,Zamora Chinchipe,1,2,7.64
209,Orellana,7,2,6.97


In [35]:
# Aplicación de Filtros
numero_minimo_empresas = 20
lq_altamente_conentrado =1.5

location_quotient_set_10 = location_quotient_set_10[(location_quotient_set_10['Nro Empresas'] >= numero_minimo_empresas) & 
                                                                          (location_quotient_set_10['LQ'] >= lq_altamente_conentrado)]
print(location_quotient_set_10.shape)

location_quotient_set_10 = location_quotient_set_10.sort_values(by=['PROVINCIA', 'LQ'], ascending=[True, True])

location_quotient_set_10.head(20)

(17, 4)


,PROVINCIA,CLUSTERS_KMEANS_SET_10,Nro Empresas,LQ
20,Bolivar,0,32,8.59
23,Carchi,0,48,1.68
31,Cañar,0,97,3.67
39,Chimborazo,0,222,3.40
72,El Oro,9,68,1.54
70,El Oro,6,44,1.85
104,Galápagos,11,27,2.30
137,Imbabura,11,46,1.64
146,Loja,5,76,1.85
143,Loja,0,267,2.29


In [36]:
# Realizamos el MERGE entre los dos datasets por las columnas 'Provincia' y 'CIIU_NIVEL_4_DESC'
merged_df = pd.merge(location_quotient_set_10,
                     pd_companias[['RUC', 'PROVINCIA', 'CLUSTERS_KMEANS_SET_10', 'CIIU_NIVEL_6_DESC', 'SEGMENTO', 'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'UTILIDAD_NETA_2023']],                       
                     on=['PROVINCIA', 'CLUSTERS_KMEANS_SET_10'], 
                     how='inner')


grouped_df = merged_df.groupby(['PROVINCIA', 'CLUSTERS_KMEANS_SET_10', 'Nro Empresas', 'LQ'], 
                               as_index=False)[['INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 
                                                'UTILIDAD_NETA_2023']].mean()

grouped_df = grouped_df.rename(columns={'INGRESOS_ACTIVIDADES_ORDINARIAS_2023': 'INGRESOS PROMEDIO',
                                        'UTILIDAD_NETA_2023': 'UTILIDAD PROMEDIO'})

grouped_df = grouped_df[grouped_df['UTILIDAD PROMEDIO'] > 0]

grouped_df.shape
grouped_df


,PROVINCIA,CLUSTERS_KMEANS_SET_10,Nro Empresas,LQ,INGRESOS PROMEDIO,UTILIDAD PROMEDIO
3,Chimborazo,0,222,3.40,19558.95,406.41
5,El Oro,9,68,1.54,2094661.87,11109.01
6,Galápagos,11,27,2.30,855755.97,14310.19
7,Imbabura,11,46,1.64,506206.99,5802.13
9,Loja,5,76,1.85,232364.77,1524.48
13,Santa Elena,9,28,2.28,2118626.79,26883.86
15,Sucumbíos,5,31,3.46,254777.59,5205.65


In [37]:

# Iterar cada uno de los registros
for index, row in grouped_df.iterrows():

    # Filtrar por PROVINCIA y CLUSTER_DBSCAN_SET4
    df_filtrado = pd_companias[(pd_companias['PROVINCIA'] == row['PROVINCIA']) & (pd_companias['CLUSTERS_KMEANS_SET_10'] == row['CLUSTERS_KMEANS_SET_10'])]

    total_empresas = df_filtrado.shape[0] # Total Filas

     # Calcular distribución del segmento
    segment_distribution = df_filtrado['SEGMENTO'].value_counts(normalize=True) * 100
    segment_distribution_str = ', '.join([f"{seg}: {perc:.2f}%" for seg, perc in segment_distribution.items()])

    grouped_df.loc[index, 'Distribución 2 SEGMENTO'] = segment_distribution_str


    # Calcular distribución del CIIU Nivel 1
    ciiu_n1_distribution = df_filtrado['CIIU_NIVEL_1'].value_counts(normalize=True) * 100
    ciiu_n1_distribution_str = ', '.join([f"{seg}: {perc:.2f}%" for seg, perc in ciiu_n1_distribution.items()])

    grouped_df.loc[index, 'Distribución CIIU1 '] = ciiu_n1_distribution_str

    # # Agrupar por CIIU NIVEL 3
    pd_ciiu_detalle = df_filtrado.groupby(['PROVINCIA', 'CLUSTERS_KMEANS_SET_10', 'CIIU_NIVEL_3_DESC']).size().reset_index(name='TOTAL')    
    #orderna decreciente
    pd_ciiu_detalle = pd_ciiu_detalle.sort_values(by='TOTAL', ascending=False)
    # Concatenar los valores de 'CIIU_NIVEL_6_DESC' con el 'TOTAL' y separarlos por comas
    ciiu_detalle_concat = pd_ciiu_detalle.apply(lambda x: f"{x['CIIU_NIVEL_3_DESC']} ({x['TOTAL']})", axis=1).tolist()    
    # Unir todos los valores con comas
    ciiu_detalle_string = ", ".join(ciiu_detalle_concat)
    # Añadir la columna "CIIU_NIVEL_6_DESC_lista" al dataframe
    grouped_df.loc[index, 'CIIU_NIVEL_3_DESC_lista'] = ciiu_detalle_string

   
    # print(pd_ciiu_detalle[['CIIU_NIVEL_6_DESC', 'TOTAL']])

# Formato Dolares
grouped_df['INGRESOS PROMEDIO'] = grouped_df['INGRESOS PROMEDIO'].apply(lambda x: f"${x:,.2f}")
grouped_df['UTILIDAD PROMEDIO'] = grouped_df['UTILIDAD PROMEDIO'].apply(lambda x: f"${x:,.2f}")


grouped_df.head(100)

,PROVINCIA,CLUSTERS_KMEANS_SET_10,Nro Empresas,LQ,INGRESOS PROMEDIO,UTILIDAD PROMEDIO,Distribución 2 SEGMENTO,Distribución CIIU1,CIIU_NIVEL_3_DESC_lista
3,Chimborazo,0,222,3.40,"$19,558.95",$406.41,"Micro: 99.55%, Pequeña: 0.45%","N: 37.84%, M: 21.17%, J: 16.22%, C: 13.51%, A: 4.95%, I: 4.50%, K: 1.80%","ACTIVIDADES DE AGENCIAS DE VIAJES Y OPERADORES TURÍSTICOS (47), ACTIVIDADES DE SEGURIDAD PRIVADA (14), ACTIVIDADES DE ARQUITECTURA E INGENIERÍA Y ACTIVIDADES CONEXAS DE CONSULTORÍA TÉCNICA (10), TRANSMISIONES DE RADIO (9), ACTIVIDADES DE PROGRAMACIÓN INFORMÁTICA Y DE CONSULTORÍA DE INFORMÁTICA Y ACTIVIDADES CONEXAS (8), ACTIVIDADES DE CONTABILIDAD, TENEDURÍA DE LIBROS Y AUDITORIA; CONSULTORÍA FISCAL (7), OTRAS ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y TÉCNICAS N.C.P (7), PUBLICIDAD (7), ACTIVIDADES JURÍDICAS (6), ACTIVIDADES DE TELECOMUNICACIONES ALÁMBRICA (6), ACTIVIDADES DE LIMPIEZA (6), ACTIVIDADES DE CONSULTORÍA DE GESTIÓN (6), ACTIVIDADES DE SERVICIOS DE APOYO A LAS EMPRESAS N.C.P (5), ACTIVIDADES DE RESTAURANTES Y DE SERVICIO MÓVIL DE COMIDAS (5), ACTIVIDADES DE SERVICIOS DE SISTEMAS DE SEGURIDAD (5), ACTIVIDADES DE ALOJAMIENTO PARA ESTANCIAS CORTAS (5), ELABORACIÓN DE OTROS PRODUCTOS ALIMENTICIOS (5), FABRICACIÓN DE PRODUCTOS MINERALES NO METÁLICOS N.C.P (4), GANADERÍA (4), ALQUILER DE VEHÍCULOS AUTOMOTORES (4), PROCESAMIENTO DE DATOS, HOSPEDAJE Y ACTIVIDADES CONEXAS; PORTALES WEB (4), FABRICACIÓN DE PRENDAS DE VESTIR, EXCEPTO PRENDAS DE PIEL (4), CULTIVO DE PLANTAS NO PERENNES (4), ACTIVIDADES COMBINADAS DE APOYO A INSTALACIONES (3), ELABORACIÓN DE BEBIDAS (3), REPARACIÓN DE PRODUCTOS ELABORADOS DE METAL, MAQUINARIA Y EQUIPO (3), ACTIVIDADES AUXILIARES DE SEGUROS Y FONDOS DE PENSIONES (3), INSTALACIÓN DE MAQUINARIA Y EQUIPO INDUSTRIALES (3), ACTIVIDADES VETERINARIAS (2), PUBLICACIÓN DE LIBROS, PERIÓDICOS Y OTRAS ACTIVIDADES DE PUBLICACIÓN (2), FABRICACIÓN DE HOJAS DE MADERA PARA ENCHAPADO Y TABLEROS A BASE DE MADERA (2), CULTIVO DE PLANTAS PERENNES (2), OTRAS ACTIVIDADES DE TELECOMUNICACIONES (2), ACTIVIDADES DE GESTIÓN DE FONDOS (1), ACTIVIDADES DE PRODUCCIÓN DE PELÍCULAS CINEMATOGRÁFICAS, VÍDEOS Y PROGRAMAS DE TELEVISIÓN (1), ELABORACIÓN Y CONSERVACIÓN DE FRUTAS, LEGUMBRES Y HORTALIZAS (1), ELABORACIÓN DE PRODUCTOS DE MOLINERÍA, ALMIDONES Y PRODUCTOS DERIVADOS DEL ALMIDÓN (1), ELABORACIÓN Y CONSERVACIÓN DE CARNE (1), ACTIVIDADES DE TELECOMUNICACIONES INALÁMBRICAS (1), ELABORACIÓN DE ALIMENTOS PREPARADOS PARA ANIMALES (1), FABRICACIÓN DE MAQUINARIA DE USO ESPECIAL (1), EXTRACCIÓN DE MADERA (1), ENSAYOS Y ANÁLISIS TÉCNICOS (1), OTRAS ACTIVIDADES DE SERVICIOS DE INFORMACIÓN (1), INVESTIGACIONES Y DESARROLLO EXPERIMENTAL EN EL CAMPO DE LAS CIENCIAS NATURALES Y LA INGENIERÍA (1), OTRAS INDUSTRIAS MANUFACTURERAS N.C.P (1), PROGRAMACIÓN Y TRANSMISIONES DE TELEVISIÓN (1), PUBLICACIÓN DE PROGRAMAS INFORMÁTICOS (1)"
5,El Oro,9,68,1.54,"$2,094,661.87","$11,109.01","Mediana: 95.59%, Pequeña: 4.41%","A: 88.24%, C: 4.41%, M: 2.94%, J: 2.94%, N: 1.47%","ACUICULTURA (35), CULTIVO DE PLANTAS PERENNES (20), GANADERÍA (3), ACTIVIDADES DE ARQUITECTURA E INGENIERÍA Y ACTIVIDADES CONEXAS DE CONSULTORÍA TÉCNICA (1), ACTIVIDADES DE SEGURIDAD PRIVADA (1), CULTIVO DE PLANTAS NO PERENNES (1), ACTIVIDADES DE TELECOMUNICACIONES ALÁMBRICA (1), FABRICACIÓN DE HOJAS DE MADERA PARA ENCHAPADO Y TABLEROS A BASE DE MADERA (1), FUNDICIÓN DE METALES (1), IMPRESIÓN Y ACTIVIDADES DE SERVICIOS RELACIONADOS CON LA IMPRESIÓN (1), OTRAS ACTIVIDADES DE TELECOMUNICACIONES (1), OTRAS ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y TÉCNICAS N.C.P (1), PESCA (1)"
6,Galápagos,11,27,2.30,"$855,755.97","$14,310.19","Pequeña: 59.26%, Mediana: 37.04%, Micro: 3.70%","N: 62.96%, I: 25.93%, A: 7.41%, M: 3.70%","ACTIVIDADES DE AGENCIAS DE VIAJES Y OPERADORES TURÍSTICOS (13), OTROS SERVICIOS DE RESERVAS Y ACTIVIDADES CONEXAS (4), ACTIVIDADES DE RESTAURANTES Y DE SERVICIO MÓVIL DE COMIDAS (3), ACTIVIDADES DE ALOJAMIENTO PARA ESTANCIAS CORTAS (3), ACTIVIDADES DE SERVICIO DE BEBIDAS (1